In [1]:
import load_corpus
import generate_CBOW_ds
from datasets import load_dataset
import generate_two_tower_data
import pandas as pd
import sp_tokenizer
import torch
import two_tower_model
import wandb
import torch.optim as optim


In [8]:
# corpus = load_corpus.load_corpus()

In [9]:
# dataset = load_dataset("ms_marco", "v1.1")
# train_data = dataset["train"]
# train_df = pd.DataFrame(train_data)

In [10]:
# ds, passage_dict = generate_two_tower_data.generate_two_tower_data(train_df, 'ms_marco_model.model.model')

In [11]:
# torch.save(ds,'two_tower_training_data.pt')

In [2]:
ds = torch.load('two_tower_training_data.pt', map_location='cpu')

In [3]:
def custom_collate_fn(batch):
    # Sort the batch by target sequence length (in descending order)
    batch = sorted(batch, key=lambda x: len(x[0]), reverse=True)

    # Get the sequence lengths for both target and context
    target_seq_lengths = [len(item[0]) for item in batch]
    context_seq_lengths = [len(item[1]) for item in batch]

    # Pad target and context sequences separately
    padded_target_sequences = torch.nn.utils.rnn.pad_sequence([item[0] for item in batch], batch_first=True)
    padded_context_sequences = torch.nn.utils.rnn.pad_sequence([item[1] for item in batch], batch_first=True)

    # Create masks for the padded values for both target and context
    target_mask = (padded_target_sequences != 0).float()
    context_mask = (padded_context_sequences != 0).float()

    # Extract labels
    labels = [item[2] for item in batch]

    return torch.LongTensor(padded_target_sequences), torch.LongTensor(padded_context_sequences), torch.LongTensor(labels)


In [4]:
tokenizer = sp_tokenizer.sp_tokenizer()
def sp_decode(vectors,tokenizer):
    print(tokenizer.decode(vectors[0].tolist()), '\n',tokenizer.decode(vectors[1].tolist()),'\n', vectors[2].tolist())

In [6]:
# ds_part = ds[:20000]
# torch.save(ds_part, 'two_tower_training_section_of_data.pt')
# ds_part = torch.load('two_tower_training_section_of_data.pt')
# ds_part = torch.load('two_tower_training_section_of_data.pt')
ds = torch.load('two_tower_training_data.pt', map_location = 'cpu')
batch_size = 512
dl = torch.utils.data.DataLoader(ds, batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn)


In [7]:
vocab_size = 16000  # Vocabulary size
embedding_dim = 100  # Dimension of word embeddings
hidden_dim = 128  # GRU hidden state size
output_dim = 64  # Desired output embedding dimension

In [8]:
model_GRU = two_tower_model.GRUEmbeddingModel(vocab_size, embedding_dim, hidden_dim, output_dim)
model_LSTM = two_tower_model.LSTMEmbeddingModel(vocab_size, embedding_dim, hidden_dim, output_dim)
cosine_similarity_loss = two_tower_model.CosineSimilarityLoss(margin = 0.2)
optimizer = optim.SGD(list(model_GRU.parameters()) + list(model_LSTM.parameters()), lr=0.01)


In [9]:
# import torch.nn.functional as F

# for query_0, doc_0, label_0 in dl:

#     query_emb = model_GRU(query_0)
#     doc_emb = model_LSTM(doc_0)
#     loss,predictions = cosine_similarity_loss(query_emb,doc_emb,label_0)
#     cos_sim = F.cosine_similarity(query_emb, doc_emb)
#     print(query_emb.size(), '\n', doc_emb.size(),'\n',loss,'\n',predictions, '\n',cos_sim)
#     break

In [10]:
# import torch.nn as nn
# embd_unit = nn.Embedding(vocab_size, embedding_dim)
# gru_unit = nn.GRU(embedding_dim,hidden_dim)
# fc_unit = nn.Linear(hidden_dim,output_dim)
# output, hidden = gru_unit(embd_unit(query_0))
# output = fc_unit(output)
# output = torch.mean(output, dim=1)

# print(output.size(), hidden.size())

In [11]:
wandb.login(key="9091f9245cf64052fa6d4eae03190a076fd87fe8")
wandb.init(project ='mlx_search_eninge', entity = 'basharkabalan',     config={
    "learning_rate": 0.01,
    "architecture": "two_tower",
    "dataset": "ms_marco_v1.1",
    "epochs": 3,
    })



Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: basharkabalan. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\basha/.netrc


In [12]:
num_epochs = 20
correct_predictions = 0
total_predictions = 0

for epoch in range(num_epochs):
    for query, doc, label in dl:
        optimizer.zero_grad()
        query = model_GRU(query)
        doc = model_LSTM(doc)
        # query = query.view(1,-1)
        # doc = doc.view(1,-1)
        loss,predictions = cosine_similarity_loss(query,doc,label)
        correct_predictions += torch.sum(predictions==label)
        total_predictions += batch_size
        acc = correct_predictions/total_predictions
        loss.backward()
        optimizer.step()


        wandb.log({"acc":acc,"loss": loss})
    print(f"Epoch {epoch+1}/{num_epochs}, Acc: {acc}, Loss: {loss.item()}")

c:\Users\basha\Desktop\founders_and_coders\week2\search engine\to_use\polished\two_tower_model.py:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  predictions = torch.tensor(cosine_similarity>self.margin)


Epoch 1/20, Acc: 0.49969974160194397, Loss: 0.9757241010665894
Epoch 2/20, Acc: 0.49989053606987, Loss: 0.9787083864212036
Epoch 3/20, Acc: 0.49995365738868713, Loss: 1.0249922275543213
Epoch 4/20, Acc: 0.4999798536300659, Loss: 0.9374917149543762
Epoch 5/20, Acc: 0.5000004172325134, Loss: 0.9652059078216553
Epoch 6/20, Acc: 0.5000143051147461, Loss: 0.9633328914642334
Epoch 7/20, Acc: 0.5000231266021729, Loss: 1.0142943859100342
Epoch 8/20, Acc: 0.5000312328338623, Loss: 1.0624384880065918
Epoch 9/20, Acc: 0.5000429749488831, Loss: 1.015870451927185
Epoch 10/20, Acc: 0.5003427863121033, Loss: 0.9498809576034546


KeyboardInterrupt: 

In [32]:
import torch.nn.functional as F

tensor1 = torch.tensor([1, 1], dtype=torch.float32)
tensor2 = torch.tensor([-1,-1], dtype=torch.float32)

# Calculate the cosine similarity
cos_sim = F.cosine_similarity(tensor1.view(1, -1), tensor2.view(1, -1))
print("Cosine Similarity:", cos_sim.item())

Cosine Similarity: -0.9999999403953552


In [33]:
loss = 1-cos_sim
print(loss)
loss = torch.clamp(loss, min = 0.0)
print(loss)
loss = (1-1)*loss
print(loss)

tensor([2.])
tensor([2.])
tensor([0.])


In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CosineSimilarityLoss(nn.Module):
    def __init__(self, margin=0.0):
        super(CosineSimilarityLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, target):
        # Calculate cosine similarity
        cosine_similarity = F.cosine_similarity(output1, output2)
        print(cosine_similarity)

        # Calculate loss with margin
        loss = 1.0 - cosine_similarity + self.margin
        loss = torch.clamp(loss, min=0.0)  # Use ReLU to ensure positive loss

        # Multiply by target to penalize dissimilarity
        loss = target * (1-cosine_similarity) + (1-target) * (1+cosine_similarity)

        return loss.mean()

# Usage:
output1 = torch.tensor([1, 1], dtype=torch.float32)
output2 = torch.tensor([-1,1], dtype=torch.float32)
target = torch.tensor([1])  # 1.0 for similar, 0.0 for dissimilar

loss_fn = CosineSimilarityLoss(margin=0)
loss = loss_fn(output1.view(1,-1), output2.view(1,-1), target)

loss

tensor([0.])


tensor(1.)

In [75]:
cosine_similarity_loss(output1,output2,target)

(tensor(0.0013, grad_fn=<MeanBackward0>), tensor([True, True]))

In [8]:
predicted = torch.tensor([False, True,True])
labeled = torch.tensor([0,1,1])
predicted == labeled

tensor([True, True, True])

In [18]:
query[0]

tensor([ 0.1998,  0.0077,  0.1641, -0.0393, -0.2796,  0.1467, -0.0701, -0.2469,
        -0.0106, -0.0834,  0.3304,  0.2383, -0.2311,  0.3039, -0.0148,  0.0454,
         0.2372,  0.1358, -0.0518, -0.1181, -0.1718, -0.2236, -0.0664,  0.0087,
         0.2314, -0.2332,  0.2495, -0.2539, -0.1583, -0.0447,  0.0284, -0.1259,
         0.0127, -0.0088, -0.1055, -0.1773,  0.1909, -0.0240, -0.0656, -0.0695,
        -0.2995,  0.3083, -0.2944, -0.1112, -0.1542,  0.0800,  0.3763, -0.0453,
        -0.1123, -0.1467,  0.0145,  0.1619,  0.0384, -0.1002, -0.0440, -0.1239,
         0.0996, -0.1452, -0.0407,  0.1805,  0.0183, -0.2646,  0.2392,  0.2913],
       grad_fn=<SelectBackward0>)